# Recommendation in Online Retail

In [99]:
import json
import os
import platform
import sys
from urllib.request import Request, urlopen
from urllib.error import HTTPError
import pypgx as pgx

base_url = "http://graph-server:7007"
username = "graph_dev"
password = "Welcome1"

def generateToken():
    body = json.dumps({ 'username': username, 'password': password }).encode('utf8')
    headers = { 'content-type': 'application/json' }
    request = Request(base_url + '/auth/token', data=body, headers=headers)
    try:
        response = urlopen(request).read().decode('utf-8')
        return json.loads(response).get('access_token')
    except HTTPError as err:
        if err.code == 400:
            print('Authentication failed no username/password given')
        elif err.code == 401:
            print('Authentication failed invalid username/password')
        else:
            print("Server returned HTTP response code: {} for URL: {}".format(err.code, err.url))
        os._exit(1)
 
session = pgx.get_session(base_url=base_url, token=generateToken())
analyst = session.create_analyst()
print(session)

PgxSession(id: 6c2667b1-0412-4c3f-b34b-b2f257a30bae, name: python_pgx_client)


In [100]:
statement = '''
CREATE PROPERTY GRAPH "Online Retail"
  VERTEX TABLES (
    online_retail.customers
      LABEL "Customer"
      PROPERTIES (
        customer_id AS "customer_id"
      , "country"
      )
  , online_retail.products
      LABEL "Product"
      PROPERTIES (
        stock_code AS "stock_code"
      , "description"
      )
  )
  EDGE TABLES (
    online_retail.purchases_distinct
      KEY (purchase_id)
      SOURCE KEY(customer_id) REFERENCES customers
      DESTINATION KEY(stock_code) REFERENCES products
      LABEL "has_purchased"
      PROPERTIES (
        purchase_id
      )
  , online_retail.purchases_distinct AS purchases_distinct_2
      KEY (purchase_id)
      SOURCE KEY(stock_code) REFERENCES products
      DESTINATION KEY(customer_id) REFERENCES customers
      LABEL "purchased_by"
      PROPERTIES (
        purchase_id
      )
  )
'''

In [101]:
#graph.destroy()
session.prepare_pgql(statement).execute()

False

In [102]:
graph = session.get_graph("Online Retail")
graph

PgxGraph(name: Online Retail, v: 8258, e: 532452, directed: True, memory(Mb): 10)

In [103]:
graph.query_pgql(" SELECT ID(n), n.description MATCH (n:Product) LIMIT 3 ").print()

+---------------------------------------------+
| ID(n) | n.description                       |
+---------------------------------------------+
| 4339  | SET OF 3 HEART COOKIE CUTTERS       |
| 4340  | STRIPEY CHOCOLATE NESTING BOXES     |
| 4341  | incorrectly credited C550456 see 47 |
+---------------------------------------------+



In [104]:
graph.query_pgql("SELECT ID(c), ID(p), p.description FROM MATCH (c)->(p) WHERE c.customer_id = 'cust_12353'").print()

+----------------------------------------------------+
| ID(c) | ID(p) | description                        |
+----------------------------------------------------+
| 4014  | 5009  | MINI CAKE STAND WITH HANGING CAKES |
| 4014  | 5852  | NOVELTY BISCUITS CAKE STAND 3 TIER |
| 4014  | 6678  | CERAMIC CAKE STAND + HANGING CAKES |
| 4014  | 7353  | CERAMIC CAKE BOWL + HANGING CAKES  |
+----------------------------------------------------+



In [105]:
rs = graph.query_pgql("SELECT ID(c) FROM MATCH (c) WHERE c.customer_id = 'cust_12353'")
vertex = graph.get_vertex(rs.get_row(0))
graph.destroy_vertex_property_if_exists("ppr")
analyst.personalized_pagerank(graph, vertex, rank="ppr")

VertexProperty(name: ppr, type: double, graph: Online Retail)

In [106]:
graph.query_pgql("""
  SELECT ID(p), p.description, p.ppr
  FROM MATCH (p)
  WHERE LABEL(p) = 'Product'
    AND NOT EXISTS (
      SELECT *
      FROM MATCH (p)-[:purchased_by]->(c)
      WHERE c.customer_id = 'cust_12353'
    )
  ORDER BY p.ppr DESC
  LIMIT 10
""").print()

+--------------------------------------------------------------------+
| ID(p) | description                        | ppr                   |
+--------------------------------------------------------------------+
| 7991  | REGENCY CAKESTAND 3 TIER           | 0.0013483656895780102 |
| 6394  | WHITE HANGING HEART T-LIGHT HOLDER | 0.001300076481737166  |
| 5722  | STRAWBERRY CERAMIC TRINKET POT     | 0.0010642787031750636 |
| 5442  | SET OF 3 CAKE TINS PANTRY DESIGN   | 9.987259826891447E-4  |
| 4940  | PARTY BUNTING                      | 8.800446053134877E-4  |
| 7118  | SWEETHEART CERAMIC TRINKET BOX     | 8.793185974570989E-4  |
| 7518  | PACK OF 72 RETROSPOT CAKE CASES    | 7.74948580210001E-4   |
| 5299  | 60 TEATIME FAIRY CAKE CASES        | 7.561654694509065E-4  |
| 5100  | JUMBO BAG RED RETROSPOT            | 7.258904143858246E-4  |
| 6538  | ASSORTED COLOUR BIRD ORNAMENT      | 7.223349157689754E-4  |
+--------------------------------------------------------------------+



In [ ]:
SELECT *
MATCH (c1)-[e1]->(p1)<-[e2]-(c2)-[e3]->(p2)
WHERE c1.CUSTOMER_ID = 'cust_12353'
  AND p2.STOCK_CODE = 'prod_23166'
  AND ID(c1) != ID(c2)
  AND ID(p1) != ID(p2)